# Scripts Antoine

In [2]:
# Parameters
file_location = "images/5 mins_t0.jpg"
file_location = "images/20 mins_t9.jpg"
particle_pixel_size = 8

## Data Processing

In [7]:
import numpy as np
import trackpy as tp
import pandas as pd
from PIL import Image
from skimage.exposure import equalize_adapthist
import functions as fun

# Opening, preprocessing the image
img = Image.open(file_location, mode='r')
img_array = np.array(img)
img_norm = equalize_adapthist(img_array)
img_info = {'height' : np.shape(img_array)[0], 'width' : np.shape(img_norm)[1]}

# Particle detection
locations_comp = tp.locate(img_norm, 9, minmass=4, separation=9, invert=True)
locations = tp.locate(img_norm, 9, minmass=3.5, separation=5, invert=True, threshold=0.05)

# Particle pair computation
rho_ctrs, g = fun.particle_pair(locations, img_info, particle_pixel_size, rhos=np.linspace(0.01,5*particle_pixel_size, 100))

def export_g_rho(rho_ctrs, g, file_name='data.csv'):
    data = pd.DataFrame({'rho':rho_ctrs, 'g':g})
    data.to_csv(file_name)

export_g_rho(rho_ctrs, g)
print(' > {:05d} particles found'.format(len(locations)))

 > 01947 particles found


## Plotting figure with particle positions

In [8]:
from bokeh.plotting import figure, show

f = figure(tooltips=[('x', '$x'),('y', '$y'),('lum', '@image')])
f.image([1-img_norm], x=0, y=0, dw=img_info['width'], dh=img_info['height'])
f.scatter(locations_comp['x'], locations_comp['y'], marker='+', line_color='lime', size=5)
f.scatter(locations['x'], locations['y'], marker='x', line_color='red', size=5)
show(f)

## Plotting figure with $g(\rho)$

In [11]:
from bokeh.plotting import figure, show

f = figure()
f.line(rho_ctrs, g)
f.line([1,1], [0,np.max(g)], line_dash='dotted', line_color='black')
f.line(rho_ctrs, np.ones(np.size(rho_ctrs)), line_dash='dashed', line_color='black')
show(f)

## 'Mass' of particles

In [ ]:
from bokeh.plotting import figure, show

mss, bins = np.histogram(locations['mass'], bins=100)
f = figure()
f.quad(left=bins[:-1], right=bins[1:], top=mss, bottom=0*mss)
show(f)

In [13]:
locations

,y,x,mass,size,ecc,signal,raw_mass,ep
0,3.407386,39.072978,5.530075,2.121666,0.100173,0.286877,34.442606,0.048986
1,5.311453,62.909622,8.321045,2.389718,0.072846,0.301464,39.022836,0.042332
2,4.284588,85.094808,9.864021,2.490633,0.089484,0.301464,39.776163,0.041407
3,3.766847,118.071073,7.479865,2.391842,0.093227,0.264186,40.247955,0.040848
4,3.355832,133.057113,4.682411,2.278620,0.172531,0.226908,38.671511,0.042777
...,...,...,...,...,...,...,...,...
2368,507.235019,343.761470,6.923940,2.108426,0.131257,0.340362,33.606667,0.050433
2369,507.277298,361.255115,6.575474,2.230163,0.218461,0.296601,36.276591,0.046085
2370,507.658046,431.034483,3.948202,1.886452,0.043825,0.267427,35.136280,0.047847
2371,507.085736,459.160988,4.329083,2.083164,0.202592,0.215563,40.548174,0.040500
